# Tutorial: NVIDIA Canary-Qwen-2.5B Inference Pipeline

This notebook demonstrates how to run the NVIDIA Canary-Qwen-2.5B inference pipeline on a single sample file.

**What this notebook does:**
- Uses production orchestration (Azure blob storage)
- Runs Canary-Qwen-2.5B model (best accuracy)
- Processes just 1 sample file for testing
- Shows outputs (inference results, hypothesis text)

**Requirements:**
- GPU: ~10 GB VRAM
- RAM: ~5 GB system memory
- Azure credentials configured (for blob storage)

## 1. Setup: Import and Configure Parameters

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import torch
import gc

# Disable wandb for notebook runs (tutorials don't need tracking)
os.environ['WANDB_MODE'] = 'disabled'

# Set HuggingFace cache to use local models directory (avoid re-downloading)
os.environ['HF_HOME'] = str(Path.cwd().parent / "models/canary")
os.environ['TRANSFORMERS_CACHE'] = str(Path.cwd().parent / "models/canary")

# Add scripts directory to path
sys.path.insert(0, str(Path.cwd().parent / "scripts"))

from infer_canary import run

# System and GPU Memory Check
print("=" * 70)
print("Memory Status Check")
print("=" * 70)

# Check system RAM
try:
    import psutil
    mem = psutil.virtual_memory()
    ram_used = mem.used / 1024**3
    ram_total = mem.total / 1024**3
    ram_available = mem.available / 1024**3
    
    print(f"System RAM: {ram_used:.1f}/{ram_total:.1f} GB used")
    print(f"Available: {ram_available:.1f} GB")
    
    if ram_available < 10:
        print(f"⚠️  WARNING: Only {ram_available:.1f} GB RAM available")
        print(f"   Canary-Qwen needs ~5 GB free for model loading")
except ImportError:
    print("psutil not installed, skipping RAM check")

# Check GPU memory
if torch.cuda.is_available():
    print()
    for i in range(torch.cuda.device_count()):
        total = torch.cuda.get_device_properties(i).total_memory / 1024**3
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        reserved = torch.cuda.memory_reserved(i) / 1024**3
        free = total - reserved
        
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Total: {total:.2f} GB")
        print(f"  Allocated: {allocated:.2f} GB")
        print(f"  Reserved: {reserved:.2f} GB")
        print(f"  Free: {free:.2f} GB")
        
        if free < 10.0:
            print(f"  ⚠️  WARNING: Only {free:.2f} GB GPU free")
            print(f"     Canary-Qwen needs ~10 GB GPU VRAM")
else:
    print("\n⚠️  No CUDA GPU available - Canary requires GPU!")

print("=" * 70)
print("\n💡 Note: Wandb logging is disabled for tutorial runs")

## 1.1 GPU Memory Cleanup (Run if needed)

If you see "CUDA out of memory" errors, run this cell to clear GPU memory from previous runs:

In [ ]:
# Clean up GPU memory
import torch
import gc

# Clear PyTorch CUDA cache
torch.cuda.empty_cache()

# Force Python garbage collection
gc.collect()

print("✓ GPU memory cleared")

# Show updated memory status
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        reserved = torch.cuda.memory_reserved(i) / 1024**3
        total = torch.cuda.get_device_properties(i).total_memory / 1024**3
        free = total - reserved
        print(f"GPU {i}: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved, {free:.2f} GB free")

## 2. Configure Inference Parameters

Set parameters directly in Python (no yaml file needed):

In [ ]:
# Build configuration dictionary
cfg = {
    "experiment_id": "tutorial-canary-qwen-sample1",
    
    # Model settings
    "model": {
        "name": "canary",
        "path": "nvidia/canary-qwen-2.5b",  # Best accuracy model
        "device": "cuda",  # Canary requires GPU (use "cuda")
        "prompt": "Transcribe the following:",  # User prompt for the model
        "max_new_tokens": 512,  # Max tokens to generate
        "batch_size": 1  # Process one file at a time
    },
    
    # Input settings
    "input": {
        "source": "azure_blob",
        "parquet_path": "../data/raw/loc/veterans_history_project_resources_pre2010.parquet",  # Relative to notebook dir
        "blob_prefix": "loc_vhp",
        "sample_size": 1,  # Just 1 file for tutorial
        "duration_sec": 300,  # First 5 minutes only (faster)
        "sample_rate": 16000  # Canary expects 16kHz
    },
    
    # Output settings
    "output": {
        "dir": "../outputs/tutorial-canary-qwen-sample1",  # Relative to notebook dir
        "save_per_file": True  # Save individual hypothesis files
    },
    
    # Evaluation (optional)
    "evaluation": {
        "use_whisper_normalizer": True
    }
}

print("Configuration:")
print(f"  Model: {cfg['model']['path']}")
print(f"  Memory: ~5 GB RAM, ~10 GB GPU")
print(f"  Device: {cfg['model']['device']} (GPU required)")
print(f"  Sample size: {cfg['input']['sample_size']} file(s)")
print(f"  Duration: {cfg['input']['duration_sec']}s (first 5 minutes)")
print(f"  Output: {cfg['output']['dir']}")
print(f"\n⚠️  Note: Canary requires a CUDA-enabled GPU")
print(f"\n💡 Tip: First run will download the model (~5GB). Subsequent runs use cached model.")

## 3. Run Inference

This will:
1. Load 1 sample from the parquet file
2. Download audio from Azure blob storage
3. Preprocess audio (trim to 300s, convert to 16kHz mono WAV)
4. Load Canary-Qwen-2.5B model (~5GB RAM, ~10GB GPU)
5. Run transcription with speech-language model
6. Save results

**Note**: First run will download the model (~5GB). Cached after that.

In [ ]:
# Run the inference pipeline
run(cfg)

## 4. View Results

### 4.1 Inference Results (Parquet)

In [ ]:
# Load results parquet
results_path = Path(cfg['output']['dir']) / "inference_results.parquet"
df_results = pd.read_parquet(results_path)

print(f"Results shape: {df_results.shape}")
print(f"\nColumns: {list(df_results.columns)}")
print(f"\nFirst row:")
df_results.head()

### 4.2 Hypothesis Text

In [ ]:
# Display the transcription
row = df_results.iloc[0]

print("=" * 70)
print(f"File ID: {row['file_id']}")
print(f"Collection: {row['collection_number']}")
print(f"Duration: {row['duration_sec']:.1f}s")
print(f"Processing time: {row['processing_time_sec']:.1f}s")
print(f"Status: {row['status']}")
print("=" * 70)
print(f"\nTranscription (hypothesis):\n")
print(row['hypothesis'][:500] + "..." if len(row['hypothesis']) > 500 else row['hypothesis'])

### 4.3 Individual Hypothesis File

In [ ]:
# Check individual hypothesis file
hyp_file = Path(cfg['output']['dir']) / f"hyp_{row['file_id']}.txt"

if hyp_file.exists():
    print(f"Hypothesis file: {hyp_file}")
    print(f"\nContent:\n")
    print(hyp_file.read_text()[:500] + "..." if len(hyp_file.read_text()) > 500 else hyp_file.read_text())
else:
    print(f"Hypothesis file not found: {hyp_file}")

## 5. Understanding the Output

**Key output files:**
- `inference_results.parquet`: All results in structured format (hypothesis, duration, status, etc.)
- `hyp_{file_id}.txt`: Individual hypothesis text files (one per audio file)
- `hyp_canary.txt`: Combined hypothesis text (all transcriptions concatenated)
- `canary_log_*.txt`: Detailed inference log

**Key result columns:**
- `file_id`: Unique identifier
- `collection_number`: VHP collection number
- `hypothesis`: Model transcription output
- `ground_truth`: Reference transcript (for evaluation)
- `duration_sec`: Audio duration processed
- `processing_time_sec`: Time taken for inference
- `status`: success/error
- `model_name`: Model identifier

**Model used:**
- `nvidia/canary-qwen-2.5b`: 2.5B params, English with punctuation/capitalization, top OpenASR leaderboard (WER: 5.63%)

## 6. Understanding Canary-Qwen-2.5B

Canary-Qwen is a **Speech Audio Language Model (SALM)** that combines:
- **NVIDIA Canary-1B** (ASR encoder): Fast-Conformer architecture for speech recognition
- **Qwen3-1.7B** (LLM decoder): Language model with LoRA adaptation for text generation

**Key features:**
- 2.5 billion parameters total
- English speech-to-text with punctuation and capitalization
- Top performer on OpenASR leaderboard (WER: 5.63%)
- 418× RTFx (processes audio 418× faster than real-time)
- Trained on 234K hours of public speech data

**Memory requirements (tested):**
- GPU: ~10 GB VRAM
- RAM: ~5 GB system memory
- Fits on T4 16GB GPU

**Performance:**
- Better at handling degraded/archival audio than Whisper
- Preserves conversational structure and speaker patterns
- Good for oral history transcription tasks

## 7. Next Steps

To run on more files:
1. Increase `sample_size` (e.g., 10, 50, 500)
2. Set `duration_sec: None` for full audio
3. Customize the prompt: `"prompt": "Transcribe this interview:"`
4. Adjust `max_new_tokens` for longer transcriptions (default: 512, max: 1024)

To evaluate results:
```python
# This will be available after implementing evaluation
from scripts.evaluate import evaluate_results
metrics = evaluate_results(df_results, use_whisper_normalizer=True)
print(f"WER: {metrics['wer']:.2%}")
```

**Recommended config for 500-sample benchmark:**
```python
cfg = {
    "experiment_id": "vhp-canary-qwen-500",
    "model": {
        "path": "nvidia/canary-qwen-2.5b",
        "device": "cuda",
        "max_new_tokens": 1024  # Longer transcriptions
    },
    "input": {
        "sample_size": 500,  # Full benchmark
        "duration_sec": None  # Full audio (not just 5 minutes)
    },
    "output": {
        "dir": "../outputs/vhp-canary-qwen-500"
    }
}
```